In [1]:
import random
import pandas as pd

from ME_BPER import ME_BPER
from ME_IPER import extract_last_names
from ME_LOC import add_location
from ME_ORG import add_organisation

In [3]:
df = pd.read_csv("../data_aug_sources/the-middle-east-cities.csv", sep = ";", skiprows = 1)
unique_city_da = df["city_da"].drop_duplicates()
ME_LOC = [add_location(loc) for loc in unique_city_da]

In [4]:
df = pd.read_csv("../data_aug_sources/middle_eastern_organisations.csv", sep = ";", skiprows = 1)
unique_orgs = df["org"].drop_duplicates()
ME_ORG = [add_organisation(org) for org in unique_orgs]

In [2]:
ME_BPER

['Abida',
 'Tahani',
 'Raed',
 'Harun',
 'Yassin',
 'Abdullah',
 'Wafaa',
 'Nizar',
 'Saim',
 'Shirin',
 'Maha',
 'Tarik',
 'Yasmin',
 'Hamed',
 'Mehdi',
 'Maria',
 'Kassem',
 'Hatun',
 'Salma',
 'Wael',
 'Sardar',
 'Anis',
 'Nassim',
 'Wail',
 'Sina',
 'Kamal',
 'Kamran',
 'Ramla',
 'Rana',
 'Hanadi',
 'Khan',
 'Kamel',
 'Aliya',
 'Fatma',
 'Faik',
 'Hamsa',
 'Arzu',
 'Ubah',
 'Nabil',
 'Hawraa',
 'Anas',
 'Amar',
 'Junes',
 'Zamzam',
 'Rim',
 'Sheikh',
 'Zahra',
 'Arshad',
 'Sultan',
 'Sahar',
 'Sabah',
 'Nadir',
 'Saba',
 'Ben',
 'Fayez',
 'Intisar',
 'Walaa',
 'Yahya',
 'Ala',
 'Najla',
 'Imad',
 'Muna',
 'Nuri',
 'Samia',
 'Jasmin',
 'Sajjad',
 'Asima',
 'Ashraf',
 'Nasreen',
 'Aftab',
 'Taghrid',
 'Yamina',
 'Fatiha',
 'Mahir',
 'Soraya',
 'Nahla',
 'Sadika',
 'Najia',
 'Damir',
 'Mohamad',
 'Dawood',
 'Yakup',
 'Radia',
 'Samar',
 'Sami',
 'Hafsa',
 'Najat',
 'Ferhat',
 'Dalal',
 'Najiba',
 'Hossein',
 'Yasmina',
 'Shahrzad',
 'Adeel',
 'Zakaria',
 'Aminah',
 'Arif',
 'Mushtaq',

In [7]:
ME_IPER = extract_last_names("../data_aug_sources/middle_eastern_last_names.txt")
print(ME_IPER)

['Mohamed', 'Hussein', 'Ali', 'Hassan', 'Kazem', 'Abbas', 'Jassim', 'Abdel', 'Ahmed', 'Abdullah', 'Salman', 'Ibrahim', 'Saleh', 'Mahmoud', 'Khalaf', 'Hamid', 'Mahdi', 'Mohsen', 'Khudair', 'Karim', 'Gabr', 'Alwan', 'Jabbar', 'Aboud', 'Obaid', 'Rashid', 'Jawad', 'Nasser', 'Hashem', 'Dawood', 'Ismail', 'Hadi', 'Majid', 'Hamad', 'Fadel', 'Yousef', 'Saeed', 'Abdelhossein', 'Hamoud', 'Mousa', 'Khalil', 'Radi', 'Hamza', 'Qasim', 'Farhan', 'Attieh', 'Hammadi', 'Aziz', 'Lafet', 'Faleh', 'Alawi', 'Eawda', 'Yassin', 'Najm', 'Jaber', 'Salem', 'Naima', 'Hassoun', 'Abdul karim', 'Jaafar', 'Shaker', 'Eisaa', 'Abdul razzaq', 'Abdelraza', 'Katie', 'Faraj', 'Abdelhassen', 'Rahim', 'Jamil', 'Kamel', 'Habib', 'Abdul amir', 'Awad', 'Matar', 'Mansour', 'Taher', 'Abdaly', 'Zaher', 'Mostafa', 'Taha', 'Jumah', 'Zghir', 'Naji', 'Fayad', 'Latif', 'Shehab', 'Rassin', 'Sadiq', 'Abdul jabbar', 'Idan', 'Abdel rahman', 'Sultan', 'Nayef', 'Hamoudi', 'Saad', 'Hatem', 'Khamis', 'Nasif', 'Reda', 'Zidan', 'Khalifa', 'Nour

In [ ]:
def data_aug_replace(dataset):
    for sent in dataset:
        i = 0

        while i<len(sent["tokens"]):
            tag = sent["ner_tags"][i]

            if tag == 'B-PER':
                replace = random.choice(ME_BPER)
                sent["ner_tags"][i] = replace
                ME_BPER.remove(replace)
                i+=1

            elif tag == 'I-PER':
                replace = random.choice(ME_IPER)
                sent["ner_tags"][i] = replace
                ME_IPER.remove(replace)
                i+=1

            elif tag == 'B-LOC':
                span_start = i
                span_len = 1
                i += 1
                while i < len(sent["ner_tags"]) and sent["ner_tags"][i] == "I-LOC":
                    span_len += 1
                    i += 1

                same_length_LOC = []
                for loc in ME_LOC:
                    if len(loc["tokens"]) == span_len:
                        same_length_LOC.append(loc)
                if same_length_LOC:
                    replace = random.choice(same_length_LOC)
                    sent["tokens"][span_start:span_start+span_len] = replace
                    ME_LOC.remove(replace)
            
            elif tag == 'B-ORG':
                span_start = i
                span_len = 1
                i += 1
                while i < len(sent["ner_tags"]) and sent["ner_tags"][i] == "I-ORG":
                    span_len += 1
                    i += 1

                same_length_ORG = []
                for org in ME_ORG:
                    if len(org["tokens"]) == span_len:
                        same_length_ORG.append(org)
                if same_length_ORG:
                    replace = random.choice(same_length_ORG)
                    sent["tokens"][span_start:span_start+span_len] = replace
                    ME_ORG.remove(replace)

            else:
                i+=1
